In [2]:
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
import cv2.aruco as aruco
import pandas as pd
import time
import os

In [4]:
"""
Compute undistorted frames
"""
def undistortRectify(frameR, frameL, leftMapX, leftMapY, rightMapX, rightMapY):
    undistortedL= cv2.remap(frameL, leftMapX, leftMapY, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    undistortedR= cv2.remap(frameR, rightMapX, rightMapY, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    return undistortedR, undistortedL
"""
Compute camera calibration matrices, distortion coefficients, rotation and translation matrices
"""
def loadCoefficients():
    dir=os.getcwd()
    file_path = os.path.join(dir, "../Calibration", "calibrationCoefficients.yaml")
    cv_file = cv2.FileStorage(file_path, cv2.FILE_STORAGE_READ)
    camera_matrixL = cv_file.getNode("KL").mat()
    dist_matrixL = cv_file.getNode("DL").mat()
    camera_matrixR = cv_file.getNode("KR").mat()
    dist_matrixR = cv_file.getNode("DR").mat()
    rot = cv_file.getNode("rot").mat()
    trans = cv_file.getNode("trans").mat()
    leftMapX = cv_file.getNode("sLX").mat()
    leftMapY = cv_file.getNode("sLY").mat()
    rightMapX = cv_file.getNode("sRX").mat()
    rightMapY = cv_file.getNode("sRY").mat()
    Q = cv_file.getNode("Q").mat()
    cv_file.release()
    return [camera_matrixL, dist_matrixL, camera_matrixR, dist_matrixR, rot, trans, leftMapX, leftMapY, rightMapX, rightMapY, Q]


def find_depth(right_point, left_point, frame_right, frame_left, baseline, f):
    _, width_right, _ = frame_right.shape
    height_left, width_left, _ = frame_left.shape
    frame_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY)
    frame_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY)
    Ox = int(width_left/2)
    Oy = int(height_left/2)

    ## Find focal length in pixels:
    if width_right == width_left:
         f_pixel = .14
    else:
         print('Left and right camera frames do not have the same pixel width')

    vl = left_point[1]
    ul = left_point[0]
    vr = right_point[1]
    ur = right_point[0]

    """
    Stereo Rectification
    """
    # CALCULATE THE DISPARITY:

    # Create stereo map
    stereoSGBM = cv2.StereoSGBM.create()
    stereoSGBM.setMinDisparity(70) #64
    stereoSGBM.setNumDisparities(32) #postive and divisible by 16 #32
    stereoSGBM.setBlockSize(13) # has to be odd and in range from [5,21] #11
    stereoSGBM.setP1(8 * 3 * stereoSGBM.getBlockSize() ** 2) 
    stereoSGBM.setP2(32 * 3 * stereoSGBM.getBlockSize() ** 2)
    stereoSGBM.setUniquenessRatio(10) #10
    stereoSGBM.setSpeckleWindowSize(50) #50
    stereoSGBM.setSpeckleRange(16) #16

    disparitySGBM = stereoSGBM.compute(frame_left, frame_right)
    disparity_img = disparitySGBM
    x_disp=int(ur)
    y_disp=int(vr)
    disparity = disparitySGBM[x_disp,y_disp]
    
    # CALCULATE POSITION: 
    zDepth = (baseline*f_pixel)/disparity             #Depth in [cm]
    yDepth = (baseline*f_pixel*(vl - Oy))/(f_pixel*disparity)
    xDepth = (baseline*(ul - Ox))/disparity
    return xDepth, yDepth, zDepth, disparity_img, x_disp, y_disp

In [6]:
video_number = 1
dir = os.getcwd()
video_pathl = dir + "/Videos/" + f"output_camera_nov9_left{video_number}_cropped.mp4" 
video_pathr = dir + "/Videos/" + f"output_camera_nov9_right{video_number}_cropped.mp4" 
capl = cv2.VideoCapture(video_pathl)
capr = cv2.VideoCapture(video_pathr)
B = 12.25               # Distance between the cameras [cm]
f = 3.67                # Camera lense's focal length [mm]
mtxL, distL, mtxR, distR, rot, trans, leftMapX, leftMapY, rightMapX, rightMapY, Q = loadCoefficients()

data = {'Marker': [], 'X': [], 'Y': [], 'Z': [], 'Time': []}
df = pd.DataFrame(data)
frame_count = 1

Ox = 320
Oy = 240

# Set the initial color space
color_space = 'HSV'  # or 'HSL' or 'RGB'

while True:
    success_right, frame_right = capr.read()
    success_left, frame_left = capl.read()

    # Check if either frame is None
    if frame_right is None or frame_left is None:
        break

    # Apply rectification maps
    frame_right = cv2.remap(frame_right, rightMapX, rightMapY, cv2.INTER_LINEAR)
    frame_left = cv2.remap(frame_left, leftMapX, leftMapY, cv2.INTER_LINEAR)

       # Uncomment for HSL/HSV - Comment for RGB
    if color_space == 'HSV':

        # Show the frames
        cv2.imshow("frame right", frame_right) 
        cv2.imshow("frame left", frame_left)
        succes_right, frame_right = capr.read()
        succes_left, frame_left = capl.read()

        # define range of white color in HSV 
        lower_white = np.array([0,0,140])
        upper_white = np.array([255,255,255])

        hsv_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB)
        hsv_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB)

        # Threshold the HSV image to get only white colors
        mask_right = cv2.inRange(hsv_right, lower_white, upper_white)
        frame_right = cv2.bitwise_and(frame_right,frame_right, mask= mask_right)
        mask_left = cv2.inRange(hsv_left, lower_white, upper_white)
        frame_left = cv2.bitwise_and(frame_left,frame_left, mask= mask_left)

        frame_right, frame_left = undistortRectify(frame_right, frame_left, leftMapX, leftMapY, rightMapX, rightMapY)

        gray_r = frame_right[:,:,1]
        gray_l = frame_left[:,:,1]

        bgr_r = cv2.cvtColor(frame_right, cv2.COLOR_HSV2BGR)
        bgr_l = cv2.cvtColor(frame_left, cv2.COLOR_HSV2BGR)
        gray_r = cv2.cvtColor(bgr_r, cv2.COLOR_BGR2GRAY)  # Change grayscale
        gray_l = cv2.cvtColor(bgr_l, cv2.COLOR_BGR2GRAY)  # Change grayscale

    elif color_space == 'HSL':

        # Show the frames
        cv2.imshow("frame right", frame_right) 
        cv2.imshow("frame left", frame_left)
        succes_right, frame_right = capr.read()
        succes_left, frame_left = capl.read()

        # define range of white color in HSV 
        lower_white = np.array([0,0,140])
        upper_white = np.array([255,255,255])

        hsv_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB)
        hsv_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB)
        
        # Threshold the HSV image to get only white colors
        mask_right = cv2.inRange(hsv_right, lower_white, upper_white)
        frame_right = cv2.bitwise_and(frame_right,frame_right, mask= mask_right)
        mask_left = cv2.inRange(hsv_left, lower_white, upper_white)
        frame_left = cv2.bitwise_and(frame_left,frame_left, mask= mask_left)

        frame_right, frame_left = undistortRectify(frame_right, frame_left, leftMapX, leftMapY, rightMapX, rightMapY)

        bgr_r = cv2.cvtColor(frame_right, cv2.COLOR_HSV2BGR)
        bgr_l = cv2.cvtColor(frame_left, cv2.COLOR_HSV2BGR)
        gray_r = cv2.cvtColor(bgr_r, cv2.COLOR_BGR2GRAY)  # Change grayscale
        gray_l = cv2.cvtColor(bgr_l, cv2.COLOR_BGR2GRAY)  # Change grayscale
       
        pass

    elif color_space == 'RGB':
        frame_right, frame_left = undistortRectify(frame_right, frame_left, leftMapX, leftMapY, rightMapX, rightMapY)
        gray_r = cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY)  # Change grayscale
        gray_l = cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY)  # Change grayscale
        pass

    matrix_coefficients = mtxL
    distortion_coefficients = distL

    # operations on the frame come here
    frame_count += 1

    # Show the frames
    cv2.imshow("frame right", frame_right) 
    cv2.imshow("frame left", frame_left)

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    parameters = cv2.aruco.DetectorParameters()  # Marker detection parameters

    # lists of ids and the corners beloning to each id
    corners_r, ids_r, rejected_img_points_r = aruco.detectMarkers(gray_r, aruco_dict,
                                                            parameters=parameters)
                                                            #cameraMatrix=matrix_coefficients,
                                                            #distCoeff=distortion_coefficients)
                                                            #output coordinates of the 4 corners and the ID of the marker 
    corners_l, ids_l, rejected_img_points_l = aruco.detectMarkers(gray_l, aruco_dict,
                                                            parameters=parameters)
                                                            #cameraMatrix=matrix_coefficients,
                                                            #distCoeff=distortion_coefficients)
                                                            #output coordinates of the 4 corners and the ID of the marker 

    if not success_right or not success_left:                    
        break

    else:
       
        start = time.time()
        
        # Convert the BGR image to RGB
        frame_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB)
        frame_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB)

        
        # Convert the RGB image to BGR
        frame_right = cv2.cvtColor(frame_right, cv2.COLOR_RGB2BGR)
        frame_left = cv2.cvtColor(frame_left, cv2.COLOR_RGB2BGR)


        """
        Calculating Depth
        """

        center_right = 0
        center_left = 0
  
        if (np.all(ids_r is not None) & np.all(ids_l is not None)).any():
        # if np.all(ids_r is not None) and np.all(ids_l is not None):  # If there are markers found by detector
            start_time = time.time()
            elapsed_time = time.time() - start_time
            zipped_r = zip(ids_r, corners_r)
            zipped_l = zip(ids_l, corners_l)
            
            ids_r, corners_r = zip(*(sorted(zipped_r)))
            ids_l, corners_l = zip(*(sorted(zipped_l)))
            
            axis_r = np.float32([[-0.01, -0.01, 0], [-0.01, 0.01, 0], [0.01, -0.01, 0], [0.01, 0.01, 0]]).reshape(-1, 3)
            axis_l = np.float32([[-0.01, -0.01, 0], [-0.01, 0.01, 0], [0.01, -0.01, 0], [0.01, 0.01, 0]]).reshape(-1, 3)

            for i in range(0, len(ids_l)):  # Iterate in markers

                if len(ids_l) != len(ids_r):
                    continue
                else:
                    # Estimate pose of each marker and return the values rvec and tvec---different from camera coefficients
                    rvec_r, tvec_r, markerPoints_r = aruco.estimatePoseSingleMarkers(corners_r[i], 0.04, matrix_coefficients, distortion_coefficients)
                    rvec_l, tvec_l, markerPoints_l = aruco.estimatePoseSingleMarkers(corners_l[i], 0.04, matrix_coefficients, distortion_coefficients)
            
                    x_r = int(np.mean(corners_r[i][0][:,0]))
                    x_l = int(np.mean(corners_l[i][0][:,0]))
                    y_r = int(np.mean(corners_r[i][0][:,1]))
                    y_l = int(np.mean(corners_l[i][0][:,1]))

                    center_point_right = np.array([x_r, y_r])
                    center_point_left = np.array([x_l, y_l])
                    aruco.drawDetectedMarkers(frame_right, corners_r)  # Draw A square around the markers
                    aruco.drawDetectedMarkers(frame_left, corners_l)  # Draw A square around the markers
        
                    # If no marker can be caught in one camera show text "TRACKING LOST"
                    if not np.all(ids_r is not None) or not np.all(ids_l is not None):
                        cv2.putText(frame_right, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
                        cv2.putText(frame_left, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)

                    else:
                        # Function to calculate depth of object. Outputs vector of all depths in case of several balls.
                        x3d, y3d, depth, disparity_img, x_disp, y_disp = find_depth(center_point_right, center_point_left, frame_right, frame_left, B, f)
                        disparity_img = cv2.normalize(disparity_img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                        cv2.imshow("disp",disparity_img)
                        disp_img = cv2.circle(disparity_img, (x_disp, y_disp), 20, (255, 0, 0), 10)
                        cv2.imshow("disp",disp_img)
                        cv2.putText(frame_left, "3D: (" + str(round(y3d,1)) + ', ' + str(round(x3d,1)) + ', ' + str(round(depth,1)) + ')', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),3)

                        print("Depth: ", str(round(depth,1)))

                        df2 = pd.DataFrame([[ids_r[i][0], x3d, y3d, depth, elapsed_time]], columns=['Marker', 'X', 'Y', 'Z','Time'])
                        df = pd.concat([df, df2])
                        
        end = time.time()
        totalTime = end - start

        # Show the frames
        cv2.imshow("frame right", frame_right) 
        cv2.imshow("frame left", frame_left)


        # Hit "q" to close the window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            df.to_csv('depth.csv', index=False)
            break

# Release and destroy all windows before termination
capr.release()
capl.release()
df.to_csv('depth.csv', index=False)

cv2.destroyAllWindows()

AttributeError: module 'cv2.aruco' has no attribute 'detectMarkers'